# TensorFlow and TensorNetwork Devices

This notebook compares the performance of the TensorNetwork device used with TensorFlow.
We find the speedup over autograd on default qubit to be significant and allows a new tier of networks to be optimized with relative efficieny.
Using the Tensor Network backend we are able to optimize a 20-qubit network in roughly 5 minutes on a laptop computer.

In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import time

import tensorflow as tf

from context import qnetvo as QNopt

### Setup

In [2]:
def bell_state_local_RY(settings, wires):
    qml.Hadamard(wires[0])
    qml.CNOT(wires=wires[0:2])
        
    qml.RY(settings[0], wires=wires[0])
    qml.RY(settings[1], wires=wires[1])

def static_nlocal_prepare_nodes(n):
    return [QNopt.PrepareNode(1, [2*i, 2*i + 1], bell_state_local_RY, 2) for i in range(n)]

def local_RY_measure_nodes(n):
    meas_nodes = [QNopt.MeasureNode(2, 2, [0], QNopt.local_RY, 1)]
    meas_nodes.extend([
        QNopt.MeasureNode(2, 2, [2*i + 1, 2*i + 2], QNopt.local_RY, 2) for i in range(0,n-1)
    ])
    meas_nodes.append(QNopt.MeasureNode(2, 2, [2*n-1], QNopt.local_RY, 1))
    
    return meas_nodes

def tn_optimize_nlocal_chain(n):
    print("n : ", n)

    tn_prep_nodes = static_nlocal_prepare_nodes(n)
    tn_meas_nodes = local_RY_measure_nodes(n)
    
    tn_nlocal_ansatz = QNopt.NetworkAnsatz(tn_prep_nodes, tn_meas_nodes)
    tn_nlocal_ansatz.dev = qml.device("default.tensor.tf",
        wires=tn_nlocal_ansatz.network_wires,
        representation="exact",
        contraction_method="auto"
    )
    
    tn_settings = tn_nlocal_ansatz.tf_rand_scenario_settings()

    tn_cost = QNopt.nlocal_chain_cost_22(tn_nlocal_ansatz, interface="tf", diff_method="backprop")
    tn_opt_dict = QNopt.gradient_descent(
        tn_cost,
        tn_settings,
        num_steps=15,
        step_size=0.8,
        sample_width = 5,
        interface="tf"
    )
    
    print("max score : ", tn_opt_dict["opt_score"])
    
    return tn_opt_dict  

def tf_optimize_nlocal_chain(n):
    print("n : ", n)

    tf_prep_nodes = static_nlocal_prepare_nodes(n)
    tf_meas_nodes = local_RY_measure_nodes(n)
    
    tf_nlocal_ansatz = QNopt.NetworkAnsatz(tf_prep_nodes, tf_meas_nodes)

    tf_settings = tf_nlocal_ansatz.tf_rand_scenario_settings()


    tf_cost = QNopt.nlocal_chain_cost_22(tf_nlocal_ansatz, interface="tf", diff_method="backprop")
    tf_opt_dict = QNopt.gradient_descent(
        tf_cost,
        tf_settings,
        num_steps=15,
        step_size=0.8,
        sample_width = 5,
        interface="tf"
    )
    
    print("max score : ", tf_opt_dict["opt_score"])
    
    return tf_opt_dict 

def ag_optimize_nlocal_chain(n):
    print("n : ", n)

    ag_prep_nodes = static_nlocal_prepare_nodes(n)
    ag_meas_nodes = local_RY_measure_nodes(n)
    
    ag_nlocal_ansatz = QNopt.NetworkAnsatz(ag_prep_nodes, ag_meas_nodes)
    
    ag_settings = ag_nlocal_ansatz.rand_scenario_settings()

    ag_cost = QNopt.nlocal_chain_cost_22(ag_nlocal_ansatz, interface="autograd", diff_method="backprop")
    ag_opt_dict = QNopt.gradient_descent(
        ag_cost,
        ag_settings,
        num_steps=15,
        step_size=0.8,
        sample_width = 5
    )
    
    print("max score : ", ag_opt_dict["opt_score"])
    
    return ag_opt_dict 

## Interface Comparison on Backpropagation

### Autograd

In [3]:
%%time

np.random.seed(1)
ag6_opt_dict = ag_optimize_nlocal_chain(6)

n :  6
iteration :  0 , score :  0.2910119273912153
iteration :  5 , score :  1.3328754190425522
iteration :  10 , score :  1.4111477848643927
max score :  1.4141033563290029
CPU times: user 4.19 s, sys: 151 ms, total: 4.34 s
Wall time: 4.37 s


### TensorFlow

In [4]:
%%time

np.random.seed(1)
tf6_opt_dict = tf_optimize_nlocal_chain(6)

n :  6


2021-09-29 20:07:27.887002: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


iteration :  0 , score :  tf.Tensor(0.2910119273912166, shape=(), dtype=float64)
iteration :  5 , score :  tf.Tensor(1.3328754203891258, shape=(), dtype=float64)
iteration :  10 , score :  tf.Tensor(1.4111477851022385, shape=(), dtype=float64)
max score :  tf.Tensor(1.414103356343892, shape=(), dtype=float64)
CPU times: user 24.7 s, sys: 2.04 s, total: 26.7 s
Wall time: 14.9 s


### TensorFlow with TensorNetwork Device

In [5]:
%%time

np.random.seed(1)
tn6_opt_dict = tn_optimize_nlocal_chain(6)

n :  6
iteration :  0 , score :  tf.Tensor(0.29101192739121495, shape=(), dtype=float64)
iteration :  5 , score :  tf.Tensor(1.332875420389127, shape=(), dtype=float64)
iteration :  10 , score :  tf.Tensor(1.4111477851022394, shape=(), dtype=float64)
max score :  tf.Tensor(1.4141033563438956, shape=(), dtype=float64)
CPU times: user 14.6 s, sys: 535 ms, total: 15.2 s
Wall time: 12.6 s


## Scaling TensorFlow on TensorNetwork Devices

In [6]:
%%time

np.random.seed(1)
n7_opt_dict = tn_optimize_nlocal_chain(7)

n :  7
iteration :  0 , score :  tf.Tensor(0.22747569616624647, shape=(), dtype=float64)
iteration :  5 , score :  tf.Tensor(1.3903842877476185, shape=(), dtype=float64)
iteration :  10 , score :  tf.Tensor(1.4134552295515312, shape=(), dtype=float64)
max score :  tf.Tensor(1.4141865830313491, shape=(), dtype=float64)
CPU times: user 27.1 s, sys: 1.01 s, total: 28.1 s
Wall time: 16.2 s


In [7]:
%%time

np.random.seed(1)
n8_opt_dict = tn_optimize_nlocal_chain(8)

n :  8
iteration :  0 , score :  tf.Tensor(0.09333571468401858, shape=(), dtype=float64)
iteration :  5 , score :  tf.Tensor(1.3438207937679794, shape=(), dtype=float64)
iteration :  10 , score :  tf.Tensor(1.412792647394611, shape=(), dtype=float64)
max score :  tf.Tensor(1.4141627006090363, shape=(), dtype=float64)
CPU times: user 1min 26s, sys: 3 s, total: 1min 29s
Wall time: 26 s


In [8]:
%%time

np.random.seed(3)
n9_opt_dict = tn_optimize_nlocal_chain(9)

n :  9
iteration :  0 , score :  tf.Tensor(0.06313287953314478, shape=(), dtype=float64)
iteration :  5 , score :  tf.Tensor(1.3406735018404106, shape=(), dtype=float64)
iteration :  10 , score :  tf.Tensor(1.413166243593456, shape=(), dtype=float64)
max score :  tf.Tensor(1.4141764073218084, shape=(), dtype=float64)
CPU times: user 6min 50s, sys: 24.7 s, total: 7min 14s
Wall time: 1min 7s


## TensorNetwork Outperforms Default Qubit

In [9]:
%%time

np.random.seed(3)
n9_ag_opt_dict = ag_optimize_nlocal_chain(9)

n :  9
iteration :  0 , score :  0.06313287953314589
iteration :  5 , score :  1.340673496587733
iteration :  10 , score :  1.4131662434911583
max score :  1.414176407316137
CPU times: user 3min 40s, sys: 35.1 s, total: 4min 15s
Wall time: 2min 5s


## Max "Efficient" (<10min) Optimization, $n=10$ (20 Qubits)

In [10]:
%%time

np.random.seed(1)
n10_opt_dict = tn_optimize_nlocal_chain(10)

n :  10
iteration :  0 , score :  tf.Tensor(0.045666421669793685, shape=(), dtype=float64)
iteration :  5 , score :  tf.Tensor(1.0793722535234427, shape=(), dtype=float64)
iteration :  10 , score :  tf.Tensor(1.3897130609471136, shape=(), dtype=float64)
max score :  tf.Tensor(1.4132963148318258, shape=(), dtype=float64)
CPU times: user 34min 42s, sys: 1min 2s, total: 35min 45s
Wall time: 4min 23s
